In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bsde_solver.tensor.tensor_train import TensorTrain, left_unfold, right_unfold
from bsde_solver.tensor.tensor_core import TensorCore
from bsde_solver.tensor.tensor_network import TensorNetwork

import numpy as np

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)


In [3]:
tt = TensorTrain(shape=[4, 4, 4, 4], ranks=[1, 3, 3, 3, 1])
tt.randomize()

TensorNetwork(
    core_0: TensorCore(r_0 {1}, m_1 {4}, r_1 {3}),
    core_1: TensorCore(r_1 {3}, m_2 {4}, r_2 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {3}),
    core_3: TensorCore(r_3 {3}, m_4 {4}, r_4 {1})
)

Implementation of several functions and algorithms with tensor train decomposition using the following random tensor train (matrix product state):

<center><img src="./images/test_tt.png" style="width: 40%"/></center>

In [4]:
tt

TensorNetwork(
    core_0: TensorCore(r_0 {1}, m_1 {4}, r_1 {3}),
    core_1: TensorCore(r_1 {3}, m_2 {4}, r_2 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {3}),
    core_3: TensorCore(r_3 {3}, m_4 {4}, r_4 {1})
)

### Left and Right unfoldings

Left unfolding:
<center><img src="./images/left_unfold.png" style="width: 40%"/></center>

Right unfolding:
<center><img src="./images/right_unfold.png" style="width: 40%"/></center>

In [5]:
R = right_unfold(tt[1])
L = left_unfold(tt[1])

print(R)
print(L)

core_1: TensorCore(r_1 {3}, m_2+r_2 {12})
core_1: TensorCore(r_1+m_2 {12}, r_2 {3})


### Left and Right orthogonalizations


Left orthogonalization:
<center><img src="./images/tt_left_ortho.png" style="width: 40%"/></center>

Right orthogonalization:
<center><img src="./images/tt_right_ortho.png" style="width: 40%"/></center>


In [6]:
tt1 = tt.copy()
tt2 = tt.copy()

In [7]:
tt[0].view(np.ndarray), tt1[0].view(np.ndarray), tt2[0].view(np.ndarray)

(array([[[-0.87, -0.08, -0.52],
         [ 0.16,  0.24,  0.17],
         [ 0.74, -0.96, -0.99],
         [ 0.4 , -0.06, -0.97]]]),
 array([[[-0.87, -0.08, -0.52],
         [ 0.16,  0.24,  0.17],
         [ 0.74, -0.96, -0.99],
         [ 0.4 , -0.06, -0.97]]]),
 array([[[-0.87, -0.08, -0.52],
         [ 0.16,  0.24,  0.17],
         [ 0.74, -0.96, -0.99],
         [ 0.4 , -0.06, -0.97]]]))

In [8]:
tt1.orthonormalize(mode="left")

<center><img src="./images/tt_left_ortho_id.png" style="width: 30%"/></center>

In [9]:
L = left_unfold(tt1[1]).view(np.ndarray)
L.T @ L

array([[ 1.,  0., -0.],
       [ 0.,  1., -0.],
       [-0., -0.,  1.]])

In [10]:
tt2.orthonormalize(mode="right")

<center><img src="./images/tt_right_ortho_id.png" style="width: 30%"/></center>

In [11]:
R = right_unfold(tt2[3]).view(np.ndarray)
R @ R.T

array([[ 1., -0., -0.],
       [-0.,  1.,  0.],
       [-0.,  0.,  1.]])

### Left & Right part of the tensor train

Left part :
<center><img src="./images/left_contract.png" style="width: 50%"/></center>
Right part :
<center><img src="./images/right_contract.png" style="width: 50%"/></center>

Then, we can write the tensor train as:

<center><img src="./images/left_right_repr.png" style="width: 80%"/></center>


## Alternating Least Squares (ALS) algorithm

In [12]:
from opt_einsum import contract

def retraction_operator(tt, i):
    operator = tt.extract([f'core_{j}' for j in range(tt.order) if j != i])
    return operator

In [13]:
tt3 = tt.copy()
tt3.orthonormalize(mode="right")

print(tt3)

P1 = retraction_operator(tt3, 0)
print("\nRectraction operator (1st):")
print(P1)
print(P1.contract())

TensorNetwork(
    core_0: TensorCore(r_0 {1}, m_1 {4}, r_1 {3}),
    core_1: TensorCore(r_1 {3}, m_2 {4}, r_2 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {3}),
    core_3: TensorCore(r_3 {3}, m_4 {4}, r_4 {1})
)

Rectraction operator (1st):
TensorNetwork(
    core_1: TensorCore(r_1 {3}, m_2 {4}, r_2 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {3}),
    core_3: TensorCore(r_3 {3}, m_4 {4}, r_4 {1})
)
TensorCore(r_1 {3}, m_2 {4}, m_3 {4}, m_4 {4}, r_4 {1})


In [14]:
tt3.orthonormalize(mode="left")
tt3.orthonormalize(mode="right", start=1)

P2 = retraction_operator(tt3, 1)
print("\nRectraction operator (2nd):")
print(P2)
print(P2.contract())

P2_mat = P2.contract().unfold(('r_1', 'r_2'), -1).view(np.ndarray)
(P2_mat @ P2_mat.T)


Rectraction operator (2nd):
TensorNetwork(
    core_0: TensorCore(r_0 {1}, m_1 {4}, r_1 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {3}),
    core_3: TensorCore(r_3 {3}, m_4 {4}, r_4 {1})
)
TensorCore(r_0 {1}, m_1 {4}, r_1 {3}, r_2 {3}, m_3 {4}, m_4 {4}, r_4 {1})


array([[ 1.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.],
       [ 0.,  1.,  0.,  0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0., -0.,  0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0., -0., -0.],
       [ 0., -0., -0.,  0.,  1.,  0., -0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-0.,  0., -0.,  0., -0.,  0.,  1.,  0.,  0.],
       [ 0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0.],
       [-0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  1.]])

In [15]:
tt3.orthonormalize(mode="left")
tt3.orthonormalize(mode="right", start=2)

P3 = retraction_operator(tt3, 2)
print(P3.contract())
P3_mat = P3.contract().unfold(('r_2', 'r_3'), -1).view(np.ndarray)
(P3_mat @ P3_mat.T)

TensorCore(r_0 {1}, m_1 {4}, m_2 {4}, r_2 {3}, r_3 {3}, m_4 {4}, r_4 {1})


array([[ 1.,  0.,  0., -0., -0., -0.,  0., -0.,  0.],
       [ 0.,  1.,  0., -0., -0., -0.,  0.,  0., -0.],
       [ 0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0.],
       [-0., -0., -0.,  1.,  0.,  0., -0., -0.,  0.],
       [-0., -0.,  0.,  0.,  1.,  0.,  0., -0.,  0.],
       [-0., -0., -0.,  0.,  0.,  1.,  0.,  0., -0.],
       [ 0.,  0.,  0., -0.,  0.,  0.,  1.,  0.,  0.],
       [-0.,  0.,  0., -0., -0.,  0.,  0.,  1.,  0.],
       [ 0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  1.]])

In [16]:
ttt = tt.copy()
ttt.orthonormalize(mode="left")

#### Rename indices

In [17]:
tt4 = tt.copy()
tt5 = tt4.rename("m_*", "n_*", inplace=False)
tt4, tt5

(TensorNetwork(
     core_0: TensorCore(r_0 {1}, m_1 {4}, r_1 {3}),
     core_1: TensorCore(r_1 {3}, m_2 {4}, r_2 {3}),
     core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {3}),
     core_3: TensorCore(r_3 {3}, m_4 {4}, r_4 {1})
 ),
 TensorNetwork(
     core_0: TensorCore(r_0 {1}, n_1 {4}, r_1 {3}),
     core_1: TensorCore(r_1 {3}, n_2 {4}, r_2 {3}),
     core_2: TensorCore(r_2 {3}, n_3 {4}, r_3 {3}),
     core_3: TensorCore(r_3 {3}, n_4 {4}, r_4 {1})
 ))

#### Micro-optimization

At each step of the ALS algorithm, we need to compute the following expression:

$$P_i^T A P_i 

In [18]:
def ALS(A, b, n_iter=10, ranks=None):
    shape = [10, 10, 10]
    ranks = [1, 3, 3, 1]
    tt = TensorTrain(shape, ranks)
    tt.randomize()
    tt.orthonormalize(mode="right", start=1)

    print(tt)

    def get_idx(j):
        if j == 0: indices = (b[j].indices[0], *tt[j].indices[1:])
        elif j == tt.order-1: indices = (*tt[j].indices[:-1], b[j].indices[2])
        else: indices = tt[j].indices
        return indices

    def get_idx2(j):
        if j == 0: indices = (f'r_{tt.order}', f'm_{j+1}', f'r_{j+1}', f't_{tt.order}', f'n_{j+1}', f't_{j+1}', )
        #(f't_{tt.order-1}', *tt[j].indices[1:], )
        elif j == tt.order-1: indices = (f'r_{j}', f'm_{j+1}', f'r_0', f't_{j}', f'n_{j+1}', f't_{0}', )
        else: indices = (*tt[j].indices, f't_{j}', f'n_{j+1}', f't_{j+1}', )
        return indices

    def micro_optimization(tt, j):
        P = retraction_operator(tt, j)
        P.name = 'P'
        # T = TensorNetwork(
        #     cores=[P, A, P.rename("m_*", "n_*", inplace=False).rename("r_*", "t_*", inplace=False)],
        #     names=['P^T','A','P']
        # ).contract(indices=get_idx2(j))
        # U = TensorNetwork(cores=[P, b], names=['P','b']).contract(indices=get_idx(j))

        # V = np.linalg.tensorsolve(T.view(np.ndarray), U.view(np.ndarray))
        # V = TensorCore(V, indices=get_idx(j))

        V = TensorNetwork(cores=[P, b], names=['P','b']).contract(indices=get_idx(j))
        # print(V)
        return V

    for i in range(n_iter):
        # Left half sweep
        for j in range(tt.order-1):
            # Micro optimization
            V = micro_optimization(tt, j)

            core_curr = tt.cores[f"core_{j}"]
            core_next = tt.cores[f"core_{j+1}"]

            L = left_unfold(V).view(np.ndarray)
            R = right_unfold(core_next).view(np.ndarray)

            Q, S = np.linalg.qr(L)
            W = S @ R

            tt.cores[f"core_{j}"] = TensorCore.like(Q, core_curr)
            tt.cores[f"core_{j+1}"] = TensorCore.like(W, core_next)

        # Right half sweep
        for j in range(tt.order-1, 0, -1):
            # Micro optimization
            V = micro_optimization(tt, j)

            core_prev = tt.cores[f"core_{j-1}"]
            core_curr = tt.cores[f"core_{j}"]

            L = left_unfold(core_prev).view(np.ndarray)
            R = right_unfold(V).view(np.ndarray)

            Q, S = np.linalg.qr(R.T)
            W = L @ S.T

            tt.cores[f"core_{j-1}"] = TensorCore.like(W, core_prev)
            tt.cores[f"core_{j}"] = TensorCore.like(Q.T, core_curr)

    return tt


# b = TensorTrain([4, 4, 4, 4, 4], [1, 3, 3, 3, 3, 1])
b = TensorTrain([10, 10, 10], [1, 4, 4, 1])
b.randomize()
b.rename('r_*', 't_*')
print(b)
b.orthonormalize(mode="right")

b = TensorTrain.from_tensor(np.arange(10*10*10).reshape(10, 10, 10), ranks=[1, 8, 8, 1])
b.rename('r_*', 't_*')
print(b)


a = np.random.rand(3, 4, 5, 3, 4, 5)
#.reshape(4, 4, 4, 4, 4, 4)
A = TensorCore(a, ['m_1', 'm_2', 'm_3', 'n_1', 'n_2', 'n_3'])
# print(b, A)

import time

start_time = time.time()
x = ALS(A, b, n_iter=100)
print(f"Elapsed time: {(time.time() - start_time):.5f}s")

TensorNetwork(
    core_0: TensorCore(t_0 {1}, m_1 {10}, t_1 {4}),
    core_1: TensorCore(t_1 {4}, m_2 {10}, t_2 {4}),
    core_2: TensorCore(t_2 {4}, m_3 {10}, t_3 {1})
)
TensorNetwork(
    core_0: TensorCore(t_0 {1}, m_1 {10}, t_1 {8}),
    core_1: TensorCore(t_1 {8}, m_2 {10}, t_2 {8}),
    core_2: TensorCore(t_2 {8}, m_3 {10}, t_3 {1})
)
TensorNetwork(
    core_0: TensorCore(r_0 {1}, m_1 {10}, r_1 {3}),
    core_1: TensorCore(r_1 {3}, m_2 {10}, r_2 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {10}, r_3 {1})
)
Elapsed time: 0.18500s


In [63]:
def scalar_ALS(X, b, n_iter=10, ranks=None):
    shape = tuple(X.shape[1] for _ in range(X.shape[0]))
    tt = TensorTrain(shape, ranks)
    tt.randomize()
    tt.orthonormalize(mode="right", start=1)

    def get_idx(j):
        if j == 0: indices = (A[j].indices[0], *tt[j].indices[1:])
        elif j == tt.order-1: indices = (*tt[j].indices[:-1], A[j].indices[2])
        else: indices = tt[j].indices
        return indices

    def micro_optimization(tt, j):
        P = retraction_operator(tt, j)
        # V = TensorNetwork(cores=[P, b], names=['P','b']).contract()#indices=get_idx(j))
        # print(V)
        print(P)
        for core in P.cores:
            core *= b
        return P.contract()

    for i in range(n_iter):
        # Left half sweep
        for j in range(tt.order-1):
            # Micro optimization
            V = micro_optimization(tt, j)
            print(V)

            core_curr = tt.cores[f"core_{j}"]
            core_next = tt.cores[f"core_{j+1}"]

            L = left_unfold(V).view(np.ndarray)
            R = right_unfold(core_next).view(np.ndarray)

            Q, S = np.linalg.qr(L)
            W = S @ R

            tt.cores[f"core_{j}"] = TensorCore.like(Q, core_curr)
            tt.cores[f"core_{j+1}"] = TensorCore.like(W, core_next)

        # Right half sweep
        for j in range(tt.order-1, 0, -1):
            # Micro optimization
            V = micro_optimization(tt, j)

            core_prev = tt.cores[f"core_{j-1}"]
            core_curr = tt.cores[f"core_{j}"]

            L = left_unfold(core_prev).view(np.ndarray)
            R = right_unfold(V).view(np.ndarray)

            Q, S = np.linalg.qr(R.T)
            W = L @ S.T

            tt.cores[f"core_{j-1}"] = TensorCore.like(W, core_prev)
            tt.cores[f"core_{j}"] = TensorCore.like(Q.T, core_curr)

    return tt

from bsde_solver.utils import flatten

b = 12
# Create a tensor of shape (4, 4, 4) from x with each axis with polynomial degree 1, x, x^2, x^3
d = 4
x = np.array([-1, -1, 0.5])
X = np.array([x**i for i in range(d)]).T
n = X.shape[0]

V = scalar_ALS(X, b, n_iter=100, ranks=[1, 3, 3, 1])
# Create tensor dot product of x (4, 4, 4)
# print(X)
# result = contract(*flatten([[X[i], ('a_'+str(i), )] for i in range(n)]))
# X.shape, result.shape, result

TensorNetwork(
    core_1: TensorCore(r_1 {3}, m_2 {4}, r_2 {3}),
    core_2: TensorCore(r_2 {3}, m_3 {4}, r_3 {1})
)
TensorCore(r_1 {3}, m_2 {4}, m_3 {4}, r_3 {1})


ValueError: axes don't match array

In [19]:
xx = x.contract()
bb = b.contract()
print("Reconstruction error:", np.linalg.norm(xx - bb))

Reconstruction error: 1.1886058111500251e-11


In [20]:
x, b

(TensorNetwork(
     P_core_0: TensorCore(r_0 {1}, m_1 {10}, r_1 {3}),
     P_core_1: TensorCore(r_1 {3}, m_2 {10}, r_2 {3}),
     P_core_2: TensorCore(r_2 {3}, m_3 {10}, r_3 {1})
 ),
 TensorNetwork(
     b_core_0: TensorCore(t_0 {1}, m_1 {10}, t_1 {8}),
     b_core_1: TensorCore(t_1 {8}, m_2 {10}, t_2 {8}),
     b_core_2: TensorCore(t_2 {8}, m_3 {10}, t_3 {1})
 ))

In [21]:
c = b.contract(indices=['m_1', 'm_2', 'm_3'])
y = np.linalg.tensorsolve(a, c.view(np.ndarray))#, axes=([3, 4, 5], [0, 1, 2]))
print(c.view(np.ndarray))
print(a.shape, c.view(np.ndarray).shape)
print(np.tensordot(a, y, axes=([3, 4, 5], [0, 1, 2])))

ValueError: solve1: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m)->(m) (size 1000 is different from 60)

In [ ]:
y = x.contract(indices=['m_1', 'm_2', 'm_3']).view(np.ndarray)

print(np.tensordot(a, y, axes=([3, 4, 5], [0, 1, 2])))

[[[ 4.23 -2.6   0.73  2.32  0.85]
  [-2.41  0.65 -1.73 -0.44 -1.14]
  [-0.21 -1.01 -0.25  0.3   1.81]
  [-1.5   1.93  0.35  1.87  0.73]]

 [[ 0.59  2.26  0.    0.46  2.5 ]
  [ 0.45  0.59  1.45 -0.47  2.44]
  [-3.55  1.52  0.93 -0.43 -1.65]
  [ 2.84  1.33 -0.35  0.51  5.02]]

 [[-2.19 -1.07  0.23  1.82  1.4 ]
  [-0.67  2.71  0.41 -1.38  1.55]
  [ 0.62  2.54  2.75  2.09 -1.73]
  [ 1.45 -2.55  1.54  0.74 -0.44]]]


In [ ]:
xx = x.contract()
bb = b.contract()
print("Reconstruction error:", np.linalg.norm(xx - bb))

Reconstruction error: 3.418895569988813e-15


### Tensor decomposition

In [ ]:
A = np.arange(10*10*10*10*10).reshape(10, 10, 10, 10, 10)
Att = TensorTrain.from_tensor(A, [1, 2, 2, 2, 2, 1])

print("Decomposition error:", np.linalg.norm(A - Att.contract().squeeze()))

Decomposition error: 1.139953776804176e-05


In [ ]:
Id = np.eye(4*4).reshape(4, 4, 4, 4)
I = TensorTrain.from_tensor(Id, [1, 2, 2, 2, 1])

print("Identity error:", np.linalg.norm(Id - I.contract().squeeze()))


id = TensorNetwork([TensorCore(Id, ['i', 'j', 'k', 'l'])])
U = TensorNetwork([TensorCore(np.random.randn(4, 4), ['i', 'j'])])

I.rename("m_1", "i")
I.rename("m_2", "j")
I.rename("m_3", "k")
I.rename("m_4", "l")

print(I, )

Identity error: 3.7416573867739413
TensorNetwork(
    core_0: TensorCore(r_0 {1}, i {4}, r_1 {2}),
    core_1: TensorCore(r_1 {2}, j {4}, r_2 {2}),
    core_2: TensorCore(r_2 {2}, k {4}, r_3 {2}),
    core_3: TensorCore(r_3 {2}, l {4}, r_4 {1})
)
